In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import yfinance as yf
plt.style.use('fivethirtyeight')

In [ ]:
df = yf.download("^NSEI" ,period= "10y", interval="1d",start="2015-01-25", end="2023-04-27")

In [ ]:
len(df)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize = (16,8))
plt.title('High Price History')
plt.plot(df['High'])
plt.xlabel('Date' , fontsize=18)
plt.ylabel('High Price INR (RS)', fontsize = 18)
plt.show()

In [ ]:
#Create DF using High cloumn only
data = df.filter(['High'])
dataset = data.values
training_data_len = math.ceil(len(dataset)* .8)

training_data_len

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
train_data = scaled_data[0:training_data_len , :]
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 60:
        print(x_train)
        print(y_train)
        print()

In [ ]:
x_train , y_train = np.array(x_train) , np.array(y_train)

In [ ]:
x_train = np.reshape(x_train , (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(50 , return_sequences=True, input_shape=(x_train.shape[1] , 1)))
model.add(LSTM(50 , return_sequences=False))

model.add(Dense(25))

model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam' , loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train , batch_size=1 , epochs=10)

In [ ]:
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60 , len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test,(x_test.shape[0] , x_test.shape[1],1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2 )
rmse

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date' , fontsize = 18)
plt.ylabel('High Price INR (RS)' , fontsize=18)
plt.plot(train['High'])
plt.plot(valid[['High', 'Predictions']])
plt.legend(['Train' , 'Val' , 'Predictions'] , loc= 'lower right')

In [ ]:
valid.tail()

In [ ]:
nifty_quote = yf.download("^NSEI" ,period= "10y", interval="1d",start="2015-01-25", end="2023-04-24")

new_df = nifty_quote.filter(['High'])
last_60_days = new_df[-60:].values
last_60_days_scaled = scaler.transform(last_60_days)

X_test = []

X_test.append(last_60_days_scaled)

X_test = np.array(X_test)

X_test = np.reshape(X_test ,(X_test.shape[0] , X_test.shape[1], 1))
pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
nifty_quote2 = yf.download("^NSEI" ,period= "10y", interval="1d",start="2023-04-19", end="2023-04-26")

In [ ]:
nifty_quote2

In [ ]:
valid.to_csv('close_valid.csv')